In [25]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from textblob import TextBlob as tb
import json


pattern = {'saúde' : ['programa','saúde','enfermagem','enfermeira','enfermeiro', 'hospital',  'médicos'],
               'segurança' : ['segurança','policial','policiamento','crime','criminalidade','criminoso','violência','violento',' lei',' Militar','ordem pública','pena','penal','defesa','infração','infrator','presidio','presidiário','preso','vigilância','vigilânte','investigação','investigador','investigar'],
               'educação' : ['educação',' ensino',' professor',' escolas'],
               'economia' : ['economia','econômico','produtividade','produção','mercado','indústria','indústriário','comércio','agro-economia','agricultura','agricola','serviços','setor','terciário','desenvolvimento','terceirizados','terceirização']}
    

In [68]:

class calcula_relevancia(object):

    def __init__(self, pattern):
        self.pattern = pattern         
    
    def tf(self,word,blob):
        return blob.words.count(word) / len(blob.words)

    def n_containing(self,word, bloblist):
        return sum(1 for blob in bloblist if word in blob.words)

    def idf(self,word, bloblist):
        return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

    def tfidf(self,word, blob, bloblist):
        return tf(word, blob) * idf(word, bloblist)

    def countSearch(self,page):        
        candidate = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}

        page = requests.get(page)
        soup = BeautifulSoup(page.content, 'html.parser')    

        for key, words in pattern.items():                            
            for w in words:            
                candidate[key]+=str(soup.find_all('p')).upper().count(w.upper())                       

        return soup, candidate
    
    #recebe uma lista de objetos soup de um cadidato
    #calcula idf-tf de cada palavra do dicionario pattern(contem as palavras por temas)
    #retorna um json contendo o idf-tf de cada tema 
    def calcula_idf_tf(self,soup_candidates,scores_candidate):      
        
        #fazendo cast do documento de soap pra blob.text    

        bloblist=[]
        for s in soup_candidates:
            document=tb("""""")
            for part in s.find_all('p'): document+= tb(part.getText())
            bloblist.append(document)

                #IDF TF PRA TODas as palavras de todos as paginas soup que foram passadas por parametro
        for blob in bloblist:    
            scores = {word: tfidf(word, blob, bloblist) for word in blob.words}

                #busca e acumula o idf-tf pras palavras do dicionario partern
                #conta a ocorencia das palavras
        for key, words in pattern.items():                          
            c=0
            for w in words:               
                word_in_blob_text=tb(w.strip(" ").upper())                                 
                if(word_in_blob_text in scores.keys()):#pra cada palavra no dicionario pattern  #veja se ela se encontra no idf-tf             
                    scores_candidate[key]+=scores[word_in_blob_text]
                    c+=1                

            if(c>0):scores_candidate[key]/=c #tira a média dos idf-tfs

        
        #return scores_candidate 
        return json.JSONEncoder(scores_candidate)
        


## Levi Fidelix (PSC)

In [70]:
#trocar esse site pois não mostra as propostas do candidato
candidate = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}

c=calcula_relevancia(pattern)

soup, candidate_count =c.countSearch("http://congressoemfoco.uol.com.br/eleicoes-2014/programa-de-governo-de-levy-fidelix/")
print(candidate_count)



ConnectionError: HTTPConnectionPool(host='congressoemfoco.uol.com.br', port=80): Max retries exceeded with url: /eleicoes-2014/programa-de-governo-de-levy-fidelix/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f5ec128eb00>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [72]:
soup_list=[soup]
candidate = {'saúde' : 0, 'segurança' : 0, 'educação' : 0, 'economia' : 0}

a = c.calcula_idf_tf(soup_list,candidate)


TypeError: __init__() takes 1 positional argument but 2 were given

In [57]:
print(b)

{'saúde': -0.0010005455637662989, 'segurança': -0.000577237825249788, 'educação': -0.00030786017346655354, 'economia': -0.0010159385724396266}


## Michel Temer (MDB)


In [19]:
#trocar esse site pois não mostra as propostas do candidato
countSearch("http://micheltemer.com.br/frases/")

{'economia': 1, 'educação': 0, 'saúde': 0, 'segurança': 1}

## Henrique Meirelles (PSD)

In [20]:
#trocar esse site pois não mostra as propostas do candidato
countSearch("http://investimentosenoticias.com.br/eleicoes/henrique-meirelles-candidato-a-presidencia-em-2018")

{'economia': 3, 'educação': 0, 'saúde': 0, 'segurança': 0}

## Aldo Rebelo e Beto Albuquerque (PSB)

In [21]:
#trocar esse site pois não mostra as propostas do candidato

countSearch("https://exame.abril.com.br/brasil/solidariedade-lanca-aldo-rebelo-como-pre-candidato-a-presidente/")

{'economia': 0, 'educação': 1, 'saúde': 0, 'segurança': 0}

## Valéria Monteiro (PMN)

In [22]:

countSearch("https://veja.abril.com.br/blog/veja-gente/valeria-monteiro-candidata-a-presidente-sou-uma-via-de-reacao/")

{'economia': 0, 'educação': 1, 'saúde': 0, 'segurança': 0}

In [23]:
#trocar esse site pois não mostra as propostas do candidato
countSearch("https://noticias.uol.com.br/politica/eleicoes/2018/noticias/2018/03/05/pre-candidata-ex-jornalista-da-globo-estuda-usar-bordao-no-estilo-eneas.htm")

{'economia': 0, 'educação': 3, 'saúde': 0, 'segurança': 0}

## Rodrigo Maia (DEM)

In [24]:
#trocar esse site pois não mostra as propostas do candidato
countSearch("http://www.gazetadopovo.com.br/eleicoes/2018/dem-lanca-maia-para-presidente-e-divulga-proposta-liberal-humanista-414ifo7ajuye81q47t1ay9s41")

{'economia': 0, 'educação': 1, 'saúde': 1, 'segurança': 1}

## Marina Silva (Rede)

In [25]:
countSearch("http://marinasilva.org.br/programa/#pagina-107")

{'economia': 95, 'educação': 144, 'saúde': 106, 'segurança': 62}

## Manuela D'Ávila (PC do B)

In [26]:
#trocar esse site pois não mostra as propostas do candidato
countSearch("http://bardebatom.com.br/noticia/manuela-davila-conheca-a-candidata-a-presidencia")

{'economia': 3, 'educação': 0, 'saúde': 2, 'segurança': 0}

In [27]:
#trocar esse site pois não mostra as propostas do candidato
countSearch("http://www.redebrasilatual.com.br/politica/2017/11/pre-candidata-manuela-davila-defende-consulta-para-revogar-medidas-de-temer")

{'economia': 0, 'educação': 0, 'saúde': 0, 'segurança': 0}